In [ ]:
# 실행마다 동일한 결과를 얻기 위해 케라스에 랜덤 시드를 사용하고 텐서플로 연산을 결정적으로 만듭니다.
import tensorflow as tf

tf.keras.utils.set_random_seed(42)
tf.config.experimental.enable_op_determinism()

In [ ]:
from tensorflow import keras
from sklearn.model_selection import train_test_split

(train_input, train_target), (test_input, test_target) = \
    keras.datasets.fashion_mnist.load_data()
# 데이터를 불러오고 표준화 전처리 후 훈련세트와 검증세트로 나눈다.
# 이때 함성곱 신경망은 2차원 이미지를 그대로 사용하기 때문에 일렬로 펼치지 않는다.

train_scaled = train_input.reshape(-1, 28, 28, 1) / 255.0
# 흑백 이미지이기 때문에 1차원 채널이 추가되며, 컬러 이미지는 3차원이 추가된다.
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42)

In [ ]:
model = keras.Sequential() # 먼저 Sequential 클래스 객체를 만들고

셰임 패딩과 밸리드 패딩

패딩이란 입력 배열 주위를 가상의 원소로 채우는 것을 의미한다. 예로, (4, 4)크기의 입력에 0을 1개 패딩하면 (6, 6)크기의 입력이 된다.

- 셰임 패딩: 함성곱 층의 출력 크기를 입력과 동일하게 만들기 위해 입력에 패딩을 추가하는 것이다.
- 밸리드 패딩: 패딩 없이 순수한 입력 배열에서만 합성곱을 하여 특성 맵을 만드는 것, 특성 맵의 크기가 줄어든다.
만약 패딩이 없다면 원소들이 2번 이상 커널과 계산되는 것과 달리, 네 모서리에 있는 4개의 값은 커널에 한번만 계산되게 된다. 만약 이 입력이 이미지라면 모서리에 있는 중요한 정보가 특성 맵에 잘 전달되지 않을 가능성이 높다. 반대로 가운데 있는 정보는 잘 표현된다.

풀링 : 합성곱 층에서 만든 특성 맵의 가로세로 크기를 줄이는 역할을 수행한다. 특성 맵의 개수는 줄이지 않는다.
최대 풀링 : 커널 영역에서 가장 큰 값을 고른다.
평균 풀링 : 커널 영역의 값을 평균화한다.
스트라이드 : 합성곱 층에서 필터가 입력 위를 이동하는 크기

In [ ]:
# 32개의 필터, 커널 크기 3*3, 렐루함수, 셰임패딩
model.add(keras.layers.Conv2D(32, kernel_size=3, activation='relu', # 커널의 개수를 32개로 지정하고, 커널 사이즈를 3으로 놓으면 (3, 3)크기가 된다.
                              padding='same', input_shape=(28,28,1)))

# 첫 번째 함성곱 층인 Conv2D를 추가한다.
# Conv2D() 매개변수로 커널의 개수, 커널 사이즈, 활성화 함수, 패딩, 입력 데이터 크기가 필요하다.
#

In [ ]:
model.add(keras.layers.MaxPooling2D(2))

In [ ]:
model.add(keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu',
                              padding='same'))
model.add(keras.layers.MaxPooling2D(2))

In [ ]:
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation='relu')) # 은닉층
model.add(keras.layers.Dropout(0.4)) # 40% 드롭아웃
model.add(keras.layers.Dense(10, activation='softmax'))# 출력층

In [ ]:
model.summary()

각 층의 파라미터의 개수를 계산할 수 있다.

첫 번째 합성곱 층은 32개의 필터를 가지고 있고 크기가 (3,3), 깊이가 1이다. 또 필터마다 하나의 절편이 있다. 3x3x1x32+32 = 320개의 파라미터가 있다.

두 번째 합성곱 층은 64개의 필터, 크기 (3,3), 깊이 32이다. 필터마다 하나의 절편이 존지하므로 3x3x32x64+64 = 18496개의 파라미터가 있다.

Flatten 층에서 (7,7,64) 크기의 특성 맵을 1차원으로 펼치면 (3136,)이며, 은닉층에서는 3136개가 100개의 뉴런과 연결되어야 하고, 100개의 절편이 있으므로 3136x100+100 = 313700개의 파라미터가 있다.

마지막 출력층은 100개의 특성이 10개의 뉴런과 연결되고, 10개의 절편이 있으므로 100x10+10 = 1010개의 파라미터가 있다.

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics='accuracy')

checkpoint_cb = keras.callbacks.ModelCheckpoint('best-cnn-model.h5',
                                                save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=2,
                                                  restore_best_weights=True)

history = model.fit(train_scaled, train_target, epochs=20,
                    validation_data=(val_scaled, val_target),
                    callbacks=[checkpoint_cb, early_stopping_cb])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
model.evaluate(val_scaled, val_target)

In [ ]:
plt.imshow(val_scaled[0].reshape(28, 28), cmap='gray_r')
plt.show()

In [ ]:
preds = model.predict(val_scaled[0:1])
print(preds)

In [ ]:
plt.bar(range(1, 11), preds[0])
plt.xlabel('class')
plt.ylabel('prob.')
plt.show()

In [ ]:
classes = ['티셔츠', '바지', '스웨터', '드레스', '코트',
           '샌달', '셔츠', '스니커즈', '가방', '앵클 부츠']

In [ ]:
import numpy as np
print(classes[np.argmax(preds)])

In [ ]:
test_scaled = test_input.reshape(-1, 28, 28, 1) / 255.0

In [ ]:
model.evaluate(test_scaled, test_target)